# Meteo data at Gotland

This notebook preprocess the meteo data at Gotland and convert the data from the netCDF format to the GOTM input format.

In [ ]:
import sys
import os
import numpy as np
import xarray as xr
import pandas as pd
sys.path.append("../gotmtool")
from gotmtool import *

## Preprocess data

In [ ]:
dataroot = os.environ['HOME']+'/data/Baltic_Sea'
startdate = '1995-01-01'
enddate = '2019-01-01'

File `Gotland.yaml` contains the information of different variables for conversion. Loop over all vairables, process and save the data.

In [ ]:
yamlfile = 'Gotland.yaml'
skip_value = 9999
max_value = 1e10

cfg = yaml_load(yamlfile)

for var in cfg['variables']:
    print(var)
    fname = cfg['variables'][var]['filename']
    vtype = cfg['variables'][var]['vartype']
    oname = cfg['variables'][var]['outname']
    ds = xr.load_dataset(dataroot+'/'+fname).sel(time=slice(startdate, enddate))
    time = ds.time.values
    ntime = time.size
    dttime = [pd.Timestamp(time[i]).to_pydatetime() for i in np.arange(ntime)]
    if '_FillValue' in ds.attrs:
        skipvalue = ds.attrs['_FillValue']
    elif 'missing_value' in ds.attrs:
        skipvalue = ds.attrs['missing_value']
    else:
        skipvalue = skip_value
    if vtype == 'scalar':
        # scalar variable
        vname = cfg['variables'][var]['varname']
        data = ds.data_vars[vname].values.squeeze()
        data[data>max_value] = np.nan
        dat_dump_ts(dttime, [data], oname, skip_value=skipvalue)
    elif vtype == 'meteo':    
        # meteorology data     
        #  u10, v10, air pressure at 2m (Pa),     
        #  air temperature at 2m (degC),     
        #  relative(%)/specific(kg/kg) humidity (determined by hum_method),     
        #  cloud cover     
        vname_u10 = cfg['variables'][var]['varname_u10']    
        vname_v10 = cfg['variables'][var]['varname_v10']    
        vname_slp = cfg['variables'][var]['varname_slp']    
        vname_sat = cfg['variables'][var]['varname_sat']    
        vname_hum = cfg['variables'][var]['varname_hum']    
        vname_cld = cfg['variables'][var]['varname_cld'] 
        u10 = ds.data_vars[vname_u10].values.squeeze()
        v10 = ds.data_vars[vname_v10].values.squeeze()
        slp = ds.data_vars[vname_slp].values.squeeze()
        sat = ds.data_vars[vname_sat].values.squeeze()
        hum = ds.data_vars[vname_hum].values.squeeze()
        cld = ds.data_vars[vname_cld].values.squeeze()
        dat_dump_ts(dttime, [u10, v10, slp, sat, hum, cld], oname, skip_value=skipvalue) 